<a href="https://colab.research.google.com/github/rong4ivy/AgentBench/blob/main/compare_LLM_langsmith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U  langchain langsmith openai
!pip install -U  langchain_community

In [7]:
import os
from langchain.llms import OpenAI, HuggingFaceHub
from langsmith import Client
from langsmith.evaluation import evaluate


# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"  # Replace with your OpenAI API key
# Set your Groq API key
os.environ['GROQ_API_KEY'] = "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"
os.environ['DEEPSEEK_API_KEY'] = "sk-1b22df9504bb4de8b0ecab4af9736a43"
os.environ['LLAMA_API_KEY']= "LL-NvPlk1hPGRjLyA7SFKVQYghNwI8HZujwnk4FEgUf9YEW9wI1c7Vc3szQUPDbryQ0"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"  # Update with your API key


Prepare dataset

In [18]:
# Load the CSV file into a DataFrame

from langsmith import Client
import pandas as pd

# Initialize the LangSmith client
client = Client()

# Load the CSV dataset
csv_file_path = "/Spar_test.csv"  # Replace with your CSV file path
df= pd.read_csv(csv_file_path)
# Upload the dataset
try:
    dataset = client.upload_csv(
        csv_file=csv_file_path,
        input_keys=["Story", "Question", "Candidate_Answers"],  # Adjust based on your CSV structure
        output_keys=["Answer"],
        name="Spar_human_Dataset"
    )
    print("Dataset uploaded successfully.")
except Exception as e:
    print(f"Error uploading dataset: {e}")


Dataset uploaded successfully.


In [1]:

def get_model_answer(model, question, options):
    prompt = f"{question}\nOptions: {', '.join(options)}\nAnswer:"
    response = model(prompt)
    return response.strip()  # Clean up the response

In [ ]:
def evaluate_model(models, dataset):
    results = []
    for item in dataset.inputs:
        question = item["question"]
        options = item["options"]

        for model in models:
            model_answer = get_model_answer(model, question, options)
            results.append({
                "question": question,
                "model": model.__class__.__name__,
                "model_answer": model_answer,
                "ground_truth": dataset.outputs[dataset.inputs.index(item)]["answer"],
                "correct": model_answer == dataset.outputs[dataset.inputs.index(item)]["answer"]
            })
    return results

In [ ]:
# Initialize the models
models = [
    OpenAI(model_name="text-davinci-003"),  # OpenAI GPT-3.5
    HuggingFaceHub(repo_id="gpt2"),          # Example Hugging Face model
    # Add more models as needed
]

if __name__ == "__main__":
    evaluation_results = evaluate_model(models, dataset)

    for result in evaluation_results:
        print(f"Question: {result['question']}")
        print(f"Model: {result['model']}")
        print(f"Model Answer: {result['model_answer']}")
        print(f"Ground Truth: {result['ground_truth']}")
        print(f"Correct: {result['correct']}\n")